## Пример классификации цифр в PyTorch

In [2]:
import numpy as np
import pandas as pd

import torch
import torchvision
import torch.nn as nn
from torchvision import transforms

import os
import zipfile

In [85]:
# from google.colab import drive
# drive.mount('/content/drive/', force_remount=True)
# data_folder = r'/content/drive/My Drive/data/'

# !unzip '/content/drive/My Drive/data/cian_dataset.zip' -d '/content/drive/My Drive/data/'

In [3]:
torch.cuda.is_available()

False

In [4]:
if torch.cuda.is_available():
    dev = 'cuda:0'
else:  
    dev = 'cpu'
device = torch.device(dev) 

In [5]:
transform = transforms.Compose([transforms.Resize((56,56)), transforms.ToTensor()])

train_data = torchvision.datasets.ImageFolder('C:/Users/Lenovo/Documents/train/',
                                              transform=transform)

trainset = torch.utils.data.Subset(train_data, list(range(0, len(train_data), 6)))

In [6]:
len(trainset)

9221

In [7]:
train_set, val_set = torch.utils.data.random_split(trainset, [6000, 3221])
train_data_loader = torch.utils.data.DataLoader(train_set, batch_size=256,
                                                shuffle=True, num_workers=2)
val_data_loader = torch.utils.data.DataLoader(val_set, batch_size=256,
                                              shuffle=True, num_workers=2)

class CustomNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(CustomNN, self).__init__()
        self.layer_1 = nn.Linear(input_size, hidden_size)
        self.layer_2 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()

    def forward(self, inputs):
        output_1 = self.relu(self.layer_1(inputs))
        output = self.layer_2(output_1)
        return output

model = CustomNN(25*25*3,10,1).to(device)
print(model)

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# обучение
torch.manual_seed(10)

epochs = 5

total_step = len(train_data_loader)
for epoch in range(epochs):
    for i, (images, labels) in enumerate(train_data_loader):
        images = images.to(device)
        images = images.reshape(-1, 25*25*3)
        labels = labels.reshape(len(labels), 1)
        labels = labels.type(torch.FloatTensor)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
               .format(epoch+1, epochs, i+1, total_step, loss.item()))

# оценка качества
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in val_data_loader:
        images = images.to(device)
        images = images.reshape(-1, 25*25*3)
        labels = labels.reshape(len(labels), 1)
        labels = labels.type(torch.FloatTensor)
        labels = labels.to(device)
        
        outputs = model(images)
        predicted = (torch.sigmoid(outputs.data) > 0.5).float()
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy: {} %'.format(100 * correct / total))

In [8]:
if torch.cuda.is_available():
    dev = 'cuda:0'
else:
    dev = 'cpu'
device = torch.device(dev)

In [20]:
class LeNet6(nn.Module):
    def __init__(self, num_classes):
        super(LeNet6, self).__init__()
        self.conv_layer1 = nn.Conv2d(in_channels=3, out_channels=128, kernel_size=3, padding=1)
        self.conv_layer2 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1)
        self.conv_layer3 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1)
        self.conv_layer4 = nn.Conv2d(in_channels=256, out_channels=4096, kernel_size=7, padding=0)
        self.pooling_layer1 = nn.MaxPool2d(kernel_size=2)
        self.pooling_layer2 = nn.MaxPool2d(kernel_size=2)
        self.pooling_layer3 = nn.MaxPool2d(kernel_size=2)

        self.linear_layer1 = nn.Linear(in_features=4096, out_features=1000)
        self.linear_layer2 = nn.Linear(in_features=1000, out_features=num_classes)

        self.relu = nn.ReLU()

    def forward(self, inputs):
        output_1 = self.relu(self.conv_layer1(inputs))
        output_2 = self.pooling_layer1(output_1)
        output_3 = self.relu(self.conv_layer2(output_2))
        output_4 = self.pooling_layer2(output_3)
        output_5 = self.relu(self.conv_layer3(output_4))
        output_6 = self.pooling_layer3(output_5)
        output_7 = self.relu(self.conv_layer4(output_6))
        output_8 = torch.flatten(output_7, 1)

        output_9 = self.relu(self.linear_layer1(output_8))
        output = self.linear_layer2(output_9)

        return output

In [24]:
model = LeNet6(1).to(device)
print(model)

LeNet6(
  (conv_layer1): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv_layer2): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv_layer3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv_layer4): Conv2d(256, 4096, kernel_size=(7, 7), stride=(1, 1))
  (pooling_layer1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (pooling_layer2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (pooling_layer3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (linear_layer1): Linear(in_features=4096, out_features=1000, bias=True)
  (linear_layer2): Linear(in_features=1000, out_features=1, bias=True)
  (relu): ReLU()
)


In [25]:
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [26]:
total_step = len(train_data_loader)
for epoch in range(5):
    for i, (images, labels) in enumerate(train_data_loader):

        labels = labels.reshape(len(labels), 1)
        labels = labels.type(torch.FloatTensor)
        outputs = model(images)

        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 1 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                   .format(epoch+1, 5, i+1, total_step, loss.item()))

Epoch [1/5], Step [1/24], Loss: 0.6952
Epoch [1/5], Step [2/24], Loss: 2.4152
Epoch [1/5], Step [3/24], Loss: 0.6985
Epoch [1/5], Step [4/24], Loss: 0.6908
Epoch [1/5], Step [5/24], Loss: 0.6911
Epoch [1/5], Step [6/24], Loss: 0.6881
Epoch [1/5], Step [7/24], Loss: 0.6925
Epoch [1/5], Step [8/24], Loss: 0.6828
Epoch [1/5], Step [9/24], Loss: 0.6683
Epoch [1/5], Step [10/24], Loss: 0.6926
Epoch [1/5], Step [11/24], Loss: 0.6468
Epoch [1/5], Step [12/24], Loss: 0.6409
Epoch [1/5], Step [13/24], Loss: 0.5920
Epoch [1/5], Step [14/24], Loss: 0.6589
Epoch [1/5], Step [15/24], Loss: 0.5415
Epoch [1/5], Step [16/24], Loss: 0.5801
Epoch [1/5], Step [17/24], Loss: 0.5207
Epoch [1/5], Step [18/24], Loss: 0.4881
Epoch [1/5], Step [19/24], Loss: 0.5036
Epoch [1/5], Step [20/24], Loss: 0.4597
Epoch [1/5], Step [21/24], Loss: 0.4628
Epoch [1/5], Step [22/24], Loss: 0.3883
Epoch [1/5], Step [23/24], Loss: 0.5174
Epoch [1/5], Step [24/24], Loss: 0.4497
Epoch [2/5], Step [1/24], Loss: 0.4059
Epoch [2/5

In [27]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in val_data_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()


    print('Accuracy: {} %'.format(100 * correct / total))

Accuracy: 52.56131636137845 %
